In [121]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import seaborn as sns

#Preprocessing
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
lmtsr = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
from wordcloud import WordCloud, STOPWORDS

#Sentiment Analysis
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm.notebook import tqdm
from textblob import TextBlob, Word, Blobber
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

#T-test
from scipy.stats import ttest_ind
import scipy.stats as stats
import scipy.stats

from collections import defaultdict
import nltk
from nltk.corpus import stopwords

# Load the stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

#calender
import calendar
import statsmodels.api as sm
import statsmodels.formula.api as smf

#Modelling
import json
import glob
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy
import spacy
from nltk.corpus import stopwords
# vis
import pyLDAvis
import pyLDAvis.gensim

import ast
from sklearn.cluster import KMeans
import numpy as np

import plotly.express as px
import plotly.io as pio

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matthewbaber/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [118]:
hf = pd.read_csv('/Users/matthewbaber/Library/Mobile Documents/com~apple~CloudDocs/1/Master/yr4/ARP/data/hanoi.csv')
dn = pd.read_csv('/Users/matthewbaber/Library/Mobile Documents/com~apple~CloudDocs/1/Master/yr4/ARP/data/da_nang.csv')
hcm = pd.read_csv('/Users/matthewbaber/Library/Mobile Documents/com~apple~CloudDocs/1/Master/yr4/ARP/data/hcm.csv')
ct = pd.read_csv('/Users/matthewbaber/Library/Mobile Documents/com~apple~CloudDocs/1/Master/yr4/ARP/data/cantho.csv')

#add column to each df to identify city
hf['city'] = 'Hanoi'
dn['city'] = 'Da Nang'
hcm['city'] = 'Ho Chi Minh City'
ct['city'] = 'Can Tho'

In [119]:
#concatenate all dataframes
df = pd.concat([ct, dn, hf, hcm])

def data_cleaning(df):

    #drop of nan that has a nan in links_n_review
    df = df.dropna(subset=['links_n_review', 'links_n_date'])

#split links_n_date into two columns on •
    df[['date', 'links']] = df['links_n_date'].str.split('•', expand=True)

    return df

def sorting_out_dates(df):

    #extract the year from the date column
    df['year'] = df['date'].str[-3:]

    def get_month_number(month):
        months = {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9,
            'Oct': 10,
            'Nov': 11,
            'Dec': 12
        }
        return months[month]

    #use this to map the months in the date colum to a number
    df['month'] = df['date'].str[:3].apply(get_month_number)

    #combine the month and year columns
    df['DATE'] = df['month'].astype(str) +  ':' + df['year'].astype(str)

    #drop the redundant columns
    df = df.drop(columns=['year', 'month', 'date'])

    #remove '-' from the DATE column
    df['DATE'] = df['DATE'].str.replace('-', '')

    #add '20'after : in the DATE column
    df['DATE'] = df['DATE'].str.replace(':', '/20')

    #remove empty space in the DATE column
    df['DATE'] = df['DATE'].str.replace(' ', '')

    return df

def datetime(df):
    #add in datetime to the DATE column
    df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%Y', errors='coerce')

    #reset index
    df = df.reset_index(drop=True)

    return df


df = data_cleaning(df)
df = sorting_out_dates(df)
df = datetime(df)

#delete duplicates
df = df.drop_duplicates(subset=['links_n_review_title_url'], keep='first')

timeline = df['DATE'].unique()

pivot = df.copy()

def pivot_table(pivot):
    #extract the first 4 numbers of the DATE column
    pivot['year'] = pivot['DATE'].astype(str).str[:4]

    #check in the city column if the city is Hanoi, then create a new column with 1 or 0
    pivot['Hanoi'] = np.where(pivot['city'] == 'Hanoi', 1, 0)
    pivot['Can Tho'] = np.where(pivot['city'] == 'Can Tho', 1, 0)
    pivot['Da Nang'] = np.where(pivot['city'] == 'Da Nang', 1, 0)
    pivot['Ho Chi Minh City'] = np.where(pivot['city'] == 'Ho Chi Minh City', 1, 0)

    pivot = pivot.pivot_table(index=['year'], values=['Hanoi', 'Can Tho', 'Da Nang', 'Ho Chi Minh City'], aggfunc='sum')

    return pivot

pivottable = pivot_table(pivot)
pivottable

,Can Tho,Da Nang,Hanoi,Ho Chi Minh City
year,,,,
2011,0,0,0,1
2012,0,3,0,3
2013,1,8,6,15
2014,20,24,69,82
2015,60,174,160,332
2016,64,431,415,1047
2017,82,683,747,1207
2018,70,952,1267,1948
2019,215,2043,2737,2709


In [122]:
stop_words = set(stopwords.words('english'))

def data_cleaning(df):

#clean the text in review column by removing any [~.,%/:;?_&+*=!-]
    def preprocess_text(text):
        cleaned = re.sub(r'[~.,%/:;?_&+*=!-]', '', text)
        return cleaned

    df['links_n_review'] = df['links_n_review'].apply(preprocess_text)

    #tokenization
    from nltk import word_tokenize,sent_tokenize 
    df['text_t'] = df['links_n_review'].str.lower().apply(word_tokenize)

    #stopwords removal
    df['text_t'] = df['text_t'].apply(lambda x: [word for word in x if word.isalpha() and word not in stop_words])

    #lemmatization
    df["text_lmt"] = df["text_t"].apply(lambda x: [lmtsr.lemmatize(word) for word in x])

    #stemming
    ps = PorterStemmer()
    df["text_stm"] = df["text_t"].apply(lambda x: [ps.stem(word) for word in x])

    df['text_stm'] = df['text_stm'].astype(str)
        
    df[".len_text"] = df["text_lmt"].str.len()
    return df

df.reset_index(drop=True, inplace=True)

df = data_cleaning(df)
df

,links_destination,links_page,links_n_contributions,links_n_review_title,links_n_review_title_url,links_n_review,links_n_date_written,links_n_date,city,links,DATE,text_t,text_lmt,text_stm,.len_text
0,Cai Rang Floating Market,"Showing results 201-210 of 1,186",5 contributions,Best trip ever,https://www.tripadvisor.co.uk/ShowUserReviews-...,Our family booked the trip to market via Then ...,Written 18 July 2023,Jun 2023 • Family,Can Tho,Family,2023-06-01,"[family, booked, trip, market, via, organised,...","[family, booked, trip, market, via, organised,...","['famili', 'book', 'trip', 'market', 'via', 'o...",97
1,Cai Rang Floating Market,"Showing results 201-210 of 1,186",9 contributions,Cai Rang floating market,https://www.tripadvisor.co.uk/ShowUserReviews-...,We visited Cai Rang floating market when join ...,Written 3 June 2023,May 2023 • Couples,Can Tho,Couples,2023-05-01,"[visited, cai, rang, floating, market, join, c...","[visited, cai, rang, floating, market, join, c...","['visit', 'cai', 'rang', 'float', 'market', 'j...",17
2,Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Wonderful Experience,https://www.tripadvisor.co.uk/ShowUserReviews-...,The market is very interesting with many fruit...,Written 8 May 2023,Apr 2023 • Couples,Can Tho,Couples,2023-04-01,"[market, interesting, many, fruit, boats, floa...","[market, interesting, many, fruit, boat, float...","['market', 'interest', 'mani', 'fruit', 'boat'...",31
3,Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Just amazing!!!,https://www.tripadvisor.co.uk/ShowUserReviews-...,Our guide picked up from our hotel and we got ...,Written 8 May 2023,May 2023 • Friends,Can Tho,Friends,2023-05-01,"[guide, picked, hotel, got, floating, market, ...","[guide, picked, hotel, got, floating, market, ...","['guid', 'pick', 'hotel', 'got', 'float', 'mar...",77
4,Cai Rang Floating Market,"Showing results 201-210 of 1,186",135 contributions,Good market experience,https://www.tripadvisor.co.uk/ShowUserReviews-...,Mr Toan from Boutique Homestay took us in his ...,Written 28 April 2023,Mar 2023 • Couples,Can Tho,Couples,2023-03-01,"[mr, toan, boutique, homestay, took, us, boat,...","[mr, toan, boutique, homestay, took, u, boat, ...","['mr', 'toan', 'boutiqu', 'homestay', 'took', ...",66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24965,Museum of Vietnamese History,Showing results 421-430 of 582,603 contributions,Nice museum that gives a flavour of the city,https://www.tripadvisor.co.uk/ShowUserReviews-...,Museum is set in some nice grounds and gives a...,Written 15 June 2014,Jun 2014 • Couples,Ho Chi Minh City,Couples,2014-06-01,"[museum, set, nice, grounds, gives, potted, hi...","[museum, set, nice, ground, give, potted, hist...","['museum', 'set', 'nice', 'ground', 'give', 'p...",21
24966,Museum of Vietnamese History,Showing results 421-430 of 582,111 contributions,Overview of entire known history,https://www.tripadvisor.co.uk/ShowUserReviews-...,It's a little threadbare but the collection ra...,Written 12 June 2014,May 2014 • Couples,Ho Chi Minh City,Couples,2014-05-01,"[little, threadbare, collection, ranges, diora...","[little, threadbare, collection, range, dioram...","['littl', 'threadbar', 'collect', 'rang', 'dio...",44
24967,Museum of Vietnamese History,Showing results 421-430 of 582,52 contributions,Museum,https://www.tripadvisor.co.uk/ShowUserReviews-...,This was a great place lots of historical arti...,Written 8 June 2014,May 2014 • Couples,Ho Chi Minh City,Couples,2014-05-01,"[great, place, lots, historical, artifacts, mu...","[great, place, lot, historical, artifact, muse...","['great', 'place', 'lot', 'histor', 'artifact'...",14
24968,Museum of Vietnamese History,Showing results 421-430 of 582,3 contributions,cool place,https://www.tripadvisor.co.uk/ShowUserReviews-...,Loved this placeits inexpensive and staff seem...,Written 4 June 2014,May 2014 • Friends,Ho Chi Minh City,Friends,2014-05-01,"[loved, placeit

In [42]:
#bag of words and TF-IDF
bag_of_words = vectorizer.fit_transform(df['text_stm'])

feature_names = vectorizer.get_feature_names_out()

# create a new dataframe with the bag of words representation
tf = pd.DataFrame(bag_of_words.toarray(), columns = vectorizer.get_feature_names_out())
tfidf_sum = tf.T.sum(axis=1).sort_values(ascending = False).reset_index().rename(columns={"index": "word", 0: "tfidf_sum"})
tfidf_transposed = tf.T

tfidf_sum = tf.T.sum(axis=1).sort_values(ascending = False).reset_index().rename(columns={"index": "word", 0: "tfidf_sum"})

#find the 2000 most infrequent words 
infreq_sum = pd.DataFrame(tfidf_sum[-2500:])
infrequent_words = infreq_sum['word'].tolist()
infrequent_words

['toofil',
 'flirt',
 'everywheresom',
 'sepia',
 'thishad',
 'performancey',
 'knowan',
 'ohhhhh',
 'ahhhh',
 'bookingshow',
 'trawl',
 'consolid',
 'saucer',
 'herethat',
 'oneth',
 'admissionand',
 ...]

In [123]:
more_stop = ['n', 'de', 'u', 'le', 'ly', 'lo', 'ta', 'go', 'get' 'also', 'lot', 'great',
 'see',
 'good',
 'nice',
 'go',
 'worth',
 'lot',
 'well',
 'realli',
 'one',
 'get',
 'around',
 'vietnam',
 'mani',
 'enjoy',
 'hanoi',
 'experi',
 'love',
 'take',
 'would',
 'also',
 'day',
 'look',
 'amaz',
 'like',
 'vietnames',
 'much',
 'area',
 'even',
 'us',
 'come',
 'must',
 'hour',
 'way',
 'went',
 'best',
 'back',
 'definit',
 'make',
 'want',
 'thing',
 'close',
 'saigon',
 'chi',
 'locat',
 'littl',
 'could',
 'outsid',
 'small',
 'ho',
 'find',
 'need',
 'fun',
 'still',
 'bit',
 'excel',
 'da',
 'part',
 'tri',
 'nang',
 'feel',
 'minh',
 'long',
 'use',
 'better',
 'top',
 'sure',
 'minut',
 'first',
 'open',
 'got',
 'found',
 'end',
 'help',
 'floor',
 'everyth',
 'big',
 'spend',
 'stop',
 'took',
 'made',
 'along',
 'though',
 'side',
 'full',
 'everi',
 'especi',
 'give',
 'pretti',
 'easi',
 'know',
 'right',
 'noth',
 'howev',
 'fantast',
 'expect',
 'think',
 'cool',
 'whole',
 'next',
 'danang']

In [124]:
stop_word = ['ho', 'chi', 'minh', 'saigon', 'hanoi', 'hcmc', 'nguyen', 'vietnam', 'hcm', 'vietnames', 'da', 'nang', 'also','go', 'lot', 'ok'] + more_stop

In [125]:
df = df[df['DATE'] >= '2018-01-01']

# 1. Building a Model using gensim

In [69]:
import gensim

preprocessed_texts = df['text_stm']

preprocessed_texts = preprocessed_texts[~preprocessed_texts.isna()]

# Convert string representation of list to actual list of tokens
data_words = [ast.literal_eval(text) for text in preprocessed_texts]

# Step 2: Create a dictionary
id2word = corpora.Dictionary(data_words)

stop_words = stop_word

unwanted_word_ids = [id2word.token2id[word] for word in stop_words if word in id2word.token2id]
id2word.filter_tokens(bad_ids=unwanted_word_ids)

# Step 3: Convert text data to bag-of-words format
corpus = [id2word.doc2bow(text) for text in data_words]

# Step 4: Save corpus and dictionary to disk
corpora.MmCorpus.serialize('corpus.mm', corpus)
id2word.save('dictionary.gensim')

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=40,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
27    -0.451689 -0.316027       1        1  12.645907
25    -0.527363  0.017272       2        1   6.026764
32    -0.054266 -0.492725       3        1   5.227530
11    -0.169037 -0.487374       4        1   5.123146
6      0.083981  0.542292       5        1   4.500511
15    -0.431323  0.315603       6        1   4.004691
24    -0.444309  0.157309       7        1   3.229709
28    -0.279378 -0.372542       8        1   3.204102
3      0.091402 -0.468547       9        1   3.157053
12     0.312187 -0.361631      10        1   3.000964
20     0.201584 -0.405842      11        1   2.856433
8      0.298158  0.386848      12        1   2.742861
22    -0.433451 -0.051156      13        1   2.659445
34     0.334828  0.092945      14        1   2.624372
14    -0.244763  0.440162      15        1   2.574701
39    -0.058697  0.457314      16        1   2.482060
17     0.341371 -0.254748      17        1   2.379511
5      0.428057 -0.184790      18        1   2.308513
37     0.493716 -0.041077      19        1   2.244880
31     0.372306  0.268104      20        1   2.046731
18     0.398816 -0.016038      21        1   2.026826
7      0.453314  0.155643      22        1   1.986628
10    -0.177464 -0.120494      23        1   1.903146
36    -0.359946 -0.145778      24        1   1.901059
35     0.084743 -0.294422      25        1   1.796633
9     -0.074163 -0.292589      26        1   1.720014
30    -0.240055 -0.232443      27        1   1.662931
23    -0.005091  0.206053      28        1   1.470437
2      0.107745  0.290978      29        1   1.450010
1      0.129437 -0.158631      30        1   1.221460
26     0.226873 -0.076525      31        1   1.094783
29    -0.075531  0.311680      32        1   1.043775
21     0.211980  0.169180      33        1   1.029923
38    -0.259965  0.166215      34        1   0.987505
33     0.162155  0.364638      35        1   0.981609
0     -0.218243  0.294646      36        1   0.945353
4     -0.196630  0.020692      37        1   0.918886
16    -0.095863  0.084137      38        1   0.792173
13     0.032291  0.015838      39        1   0.013484
19     0.032279  0.015829      40        1   0.013484, topic_info=          Term          Freq         Total Category  logprob  loglift
0               24842.000000  24842.000000  Default  15.0000  15.0000
4        place   8394.000000   8394.000000  Default  14.0000  14.0000
13   recommend   5645.000000   5645.000000  Default  13.0000  13.0000
126      visit   7828.000000   7828.000000  Default  12.0000  12.0000
94       peopl   4205.000000   4205.000000  Default  11.0000  11.0000
..         ...           ...           ...      ...      ...      ...
10      highli      0.003226   2493.699771  Topic40  -9.7832  -4.6465
11       photo      0.003226   1361.961417  Topic40  -9.7832  -4.0417
12       quiet      0.003226    292.529071  Topic40  -9.7832  -2.5036
13   recommend      0.003226   5645.940928  Topic40  -9.7832  -5.4637
14        room      0.003226   1505.059671  Topic40  -9.7832  -4.1416

[703 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.999949         
79       17  0.998039      abl
607      36  0.998638  absolut
426      14  0.969637   action
489       5  0.991378   actual
...     ...       ...      ...
95       19  0.993525      wow
232      33  0.989538    wrong
111      17  0.997287     year
499      25  0.991221    young
504       4  0.982927     zone

[504 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[28, 26, 33, 12, 7, 16, 25, 29, 4, 13, 21, 9, 23, 35, 15, 40, 18, 6, 38, 32, 19, 8, 11, 37, 36, 10, 31, 24, 3, 2, 27, 30, 22, 39, 34, 1, 5, 17, 14, 20])

In [112]:
data = df['links_n_review'].tolist()
data_ready = data_words

# Function to extract just the dominant topics from LDA model
def extract_dominant_topics(ldamodel, corpus, texts):
    topic_list = []
    
    for row_list in ldamodel[corpus]:
        row = row_list[0] if ldamodel.per_word_topics else row_list
        row = sorted(row, key=lambda x: x[1], reverse=True)
        
        dominant_topic = row[0][0]
        topic_list.append(dominant_topic)
        
    topics_df = pd.Series(topic_list, name='topic')
    return topics_df

# Assuming lda_model, corpus, and data_ready are already defined
topics_df = extract_dominant_topics(ldamodel=lda_model, corpus=corpus, texts=data_ready)
topics_df = pd.DataFrame(topics_df)
topics_df.head()

,topic
0,11
1,37
2,27
3,27
4,31


In [108]:
# Get the topics
num_topics = lda_model.num_topics
topics = lda_model.show_topics(num_topics=num_topics, num_words=4, formatted=False)

# Create a list to store each topic's information
topic_data = []

for topic_num, topic_terms in topics:
    words = [word for word, _ in topic_terms]
    topic_data.append({'topic': topic_num, 'Name': words})

# Create a DataFrame
topicn_name = pd.DataFrame(topic_data)

topicn_name 
topicn_name.head()

,topic,Name
0,0,"[absolut, last, stun, ca]"
1,1,"[old, fascin, collect, togeth]"
2,2,"[guid, say, bring, wall]"
3,3,"[price, pay, sell, person]"
4,4,"[attract, name, middl, strong]"


# 2. Using Bertopic LDA model

In [109]:
# Create a regular expression pattern to match the stopwords
stopwords = r'\b(?:' + '|'.join(stop_word) + r')\b'

# Remove the stopwords from the 'text' column
df['text_stm'] = df['text_stm'].str.replace(stopwords, '', regex=True)

df['text_stm'] = df['text_stm'].str.replace(" '',", '')

In [111]:
from bertopic import BERTopic

#the pre-processed text
text = df['text_stm'].tolist()

topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(text)

Batches:   0%|          | 0/605 [00:00<?, ?it/s]

2023-09-04 19:21:04,091 - BERTopic - Transformed documents to Embeddings
2023-09-04 19:21:38,923 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-09-04 19:21:47,279 - BERTopic - Clustered reduced embeddings


In [113]:
#topic_model to dataframe
topics_df = pd.DataFrame(topics, columns=['topic'])
topics_df

,topic
0,17
1,-1
2,-1
3,0
4,2
...,...
19331,51
19332,-1
19333,23
19334,1


In [114]:
#get the name of the topics
topic_info = topic_model.get_topic_info()
topicn_name = topic_info[['Name', 'Topic']]
topicn_name.rename(columns={'Topic': 'topic'}, inplace=True)
topicn_name

,Name,topic
0,-1_place_walk_visit_street,-1
1,0_museum_war_histori_exhibit,0
2,1_massag_spa_servic_staff,1
3,2_show_puppet_perform_opera,2
4,3_pagoda_buddha_ladi_statu,3
...,...,...
81,80_landmark_architectur_snobbi_clutter,80
82,81_holli_hear_mass_merri,81
83,82_friend_girlfriend_watcher_friendli,82
84,83_mountain_marbl_endem_showlov,83


# 3. Dyanmic Topic Model (DTM)

In [126]:
timeslices = df['DATE'].tolist()

In [127]:
#merge topic_df with df
df = df.reset_index(drop=True)
topics_df = topics_df.reset_index(drop=True)
new = pd.concat([df, topics_df], axis=1)
new

,links_destination,links_page,links_n_contributions,links_n_review_title,links_n_review_title_url,links_n_review,links_n_date_written,links_n_date,city,links,DATE,text_t,text_lmt,text_stm,.len_text,topic
0,Cai Rang Floating Market,"Showing results 201-210 of 1,186",5 contributions,Best trip ever,https://www.tripadvisor.co.uk/ShowUserReviews-...,Our family booked the trip to market via Then ...,Written 18 July 2023,Jun 2023 • Family,Can Tho,Family,2023-06-01,"[family, booked, trip, market, via, organised,...","[family, booked, trip, market, via, organised,...","['famili', 'book', 'trip', 'market', 'via', 'o...",97,17
1,Cai Rang Floating Market,"Showing results 201-210 of 1,186",9 contributions,Cai Rang floating market,https://www.tripadvisor.co.uk/ShowUserReviews-...,We visited Cai Rang floating market when join ...,Written 3 June 2023,May 2023 • Couples,Can Tho,Couples,2023-05-01,"[visited, cai, rang, floating, market, join, c...","[visited, cai, rang, floating, market, join, c...","['visit', 'cai', 'rang', 'float', 'market', 'j...",17,-1
2,Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Wonderful Experience,https://www.tripadvisor.co.uk/ShowUserReviews-...,The market is very interesting with many fruit...,Written 8 May 2023,Apr 2023 • Couples,Can Tho,Couples,2023-04-01,"[market, interesting, many, fruit, boats, floa...","[market, interesting, many, fruit, boat, float...","['market', 'interest', 'mani', 'fruit', 'boat'...",31,-1
3,Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Just amazing!!!,https://www.tripadvisor.co.uk/ShowUserReviews-...,Our guide picked up from our hotel and we got ...,Written 8 May 2023,May 2023 • Friends,Can Tho,Friends,2023-05-01,"[guide, picked, hotel, got, floating, market, ...","[guide, picked, hotel, got, floating, market, ...","['guid', 'pick', 'hotel', 'got', 'float', 'mar...",77,0
4,Cai Rang Floating Market,"Showing results 201-210 of 1,186",135 contributions,Good market experience,https://www.tripadvisor.co.uk/ShowUserReviews-...,Mr Toan from Boutique Homestay took us in his ...,Written 28 April 2023,Mar 2023 • Couples,Can Tho,Couples,2023-03-01,"[mr, toan, boutique, homestay, took, us, boat,...","[mr, toan, boutique, homestay, took, u, boat, ...","['mr', 'toan', 'boutiqu', 'homestay', 'took', ...",66,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19331,Ho Chi Minh City's Book Street,Showing results 391-397 of 397,66 contributions,Gorgeous laneway,https://www.tripadvisor.co.uk/ShowUserReviews-...,Directly across from our hotel Intercontinenta...,Written 30 January 2018,Jan 2018 • Family,Ho Chi Minh City,Family,2018-01-01,"[directly, across, hotel, intercontinental, ho...","[directly, across, hotel, intercontinental, ho...","['directli', 'across', 'hotel', 'intercontinen...",31,51
19332,Ho Chi Minh City's Book Street,Showing results 391-397 of 397,251 contributions,A lovely oasis around the corner from Notre Dame,https://www.tripadvisor.co.uk/ShowUserReviews-...,A lovely bit of cool in the heart of Saigon Ov...,Written 29 January 2018,Jan 2018 • Family,Ho Chi Minh City,Family,2018-01-01,"[lovely, bit, cool, heart, saigon, overhanging...","[lovely, bit, cool, heart, saigon, overhanging...","['love', 'bit', 'cool', 'heart', 'saigon', 'ov...",43,-1
19333,Ho Chi Minh City's Book Street,Showing results 391-397 of 397,115 contributions,Worth a trip,https://www.tripadvisor.co.uk/ShowUserReviews-...,I bought a second hand book here English The ...,Written 16 January 2018,Jan 2018 • Friends,Ho Chi Minh City,Friends,2018-01-01,"[bought, second, hand, book, english, coffee, ...","[bought, second, hand, book, english, coffee, ...","['bought', 'second', 'hand', 'book', 'english'...",15,23
19334,Ho Chi Minh City's Book Street,Showing results 391-397 of 397,156 contributions,Keep calm & read a book,https://www.tripadvisor.co.uk/ShowUserReviews-...,Nice place to read a book drink a coffee Very...,Written 8 January 2018,Jan 2018 • Frie

In [128]:
# Create an empty dictionary to store the data slices dataframes
time_topic_slices = {}

# Get unique time and topic values
unique_times = new['DATE'].unique()
unique_topics = new['topic'].unique()

# Split the DataFrame based on the time and topic slices
for t in unique_times:
    time_topic_slices[t] = {}
    df_time_filtered = new[new['DATE'] == t]
    
    for topic in unique_topics:
        df_time_topic_filtered = df_time_filtered[df_time_filtered['topic'] == topic]
        
        if not df_time_topic_filtered.empty:
            time_topic_slices[t][topic] = df_time_topic_filtered

In [129]:
# Initialize an empty DataFrame to store the results
dtm = pd.DataFrame()

# Initialize a counter for the 'cluster' column
process_counter = 0

# Loop through the nested dictionary to access each DataFrame
for time_key, topic_dict in time_topic_slices.items():
    for topic_key, df in topic_dict.items():

        # Initialize an empty string to store the concatenated result
        concatenated_string = ''

        # Loop through the DataFrame to concatenate 'text_stm'
        for i in range(len(df)):
            concatenated_string += df.iloc[i]['text_stm'] + ' '

        # Remove the trailing space at the end
        concatenated_string = concatenated_string.rstrip()

        # Create a DataFrame to append
        data_to_append = pd.DataFrame({
            'date': [df['DATE'].iloc[0]],
            'keywords': [df['Keywords'].iloc[0] if 'Keywords' in df.columns else None],
            'string': [concatenated_string],
            'topic': [df['topic'].iloc[0] if 'topic' in df.columns else None],
            'cluster': [process_counter],
            'frequency': len(df)
        })

        # Append data to the empty DataFrame
        dtm = dtm.append(data_to_append, ignore_index=True)

        # Increment the counter
        process_counter += 1

In [130]:
dtm

,date,keywords,string,topic,cluster,frequency
0,2023-06-01,None,"['famili', 'book', 'trip', 'market', 'via', 'o...",17,0,4
1,2023-06-01,None,"['awesom', 'place', 'spend', 'half', 'day', 's...",-1,1,153
2,2023-06-01,None,"['will', 'spend', 'entranc', 'fee', 'plu', 'tr...",0,2,29
3,2023-06-01,None,"['long', 'bridg', 'span', 'river', 'da', 'nang...",2,3,19
4,2023-06-01,None,"['quit', 'big', 'area', 'mani', 'sceneri', 'ph...",1,4,20
...,...,...,...,...,...,...
2146,2022-02-01,None,"['month', 'ago', 'went', 'place', 'nice', 'bea...",69,2146,1
2147,2022-02-01,None,"['great', 'street', 'nice', 'stroll', 'see', '...",36,2147,1
2148,2021-05-01,None,"['hanoi', 'old', 'quarter', 'good', 'place', '...",-1,2148,3
2149,2021-05-01,None,"['good', 'place', 'discov', 'enjoy', 'walk', '...",3,2149,1


In [131]:
dtm_model = pd.merge(dtm, topicn_name, on='topic', how='left')
dtm_model

,date,keywords,string,topic,cluster,frequency,Name
0,2023-06-01,None,"['famili', 'book', 'trip', 'market', 'via', 'o...",17,0,4,17_book_street_cafe_coffe
1,2023-06-01,None,"['awesom', 'place', 'spend', 'half', 'day', 's...",-1,1,153,-1_place_walk_visit_street
2,2023-06-01,None,"['will', 'spend', 'entranc', 'fee', 'plu', 'tr...",0,2,29,0_museum_war_histori_exhibit
3,2023-06-01,None,"['long', 'bridg', 'span', 'river', 'da', 'nang...",2,3,19,2_show_puppet_perform_opera
4,2023-06-01,None,"['quit', 'big', 'area', 'mani', 'sceneri', 'ph...",1,4,20,1_massag_spa_servic_staff
...,...,...,...,...,...,...,...
2146,2022-02-01,None,"['month', 'ago', 'went', 'place', 'nice', 'bea...",69,2146,1,69_street_cross_pavement_road
2147,2022-02-01,None,"['great', 'street', 'nice', 'stroll', 'see', '...",36,2147,1,36_bar_street_bui_vien
2148,2021-05-01,None,"['hanoi', 'old', 'quarter', 'good', 'place', '...",-1,2148,3,-1_place_walk_visit_street
2149,2021-05-01,None,"['good', 'place', 'discov', 'enjoy', 'walk', '...",3,2149,1,3_pagoda_buddha_ladi_statu


In [132]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np

# Sample data
documents = dtm_model['string'].tolist()

# Define classes for the documents
clusters = dtm_model['cluster'].tolist()

# Calculate term frequency
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

# Calculate CTF
cluster_counts = np.bincount(clusters)
ctf = np.zeros_like(X.toarray(), dtype=float)
for i, cluster_id in enumerate(clusters):
    ctf[cluster_id] += X.toarray()[i] / cluster_counts[cluster_id]

# Calculate IDF
transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
transformer.fit(X)
idf = transformer.idf_

# Calculate CTF-IDF
ctf_idf = ctf * idf

In [136]:
# Extract feature names (words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Function to get top N words for each class
def get_top_words(matrix, feature_names, top_n=10):
    top_words = {}
    for i, row in enumerate(matrix):
        top_indices = row.argsort()[-top_n:][::-1]
        top_words[i] = [feature_names[idx] for idx in top_indices]
    return top_words

# Get top 10 words for each class based on CTF-IDF values
top_words_per_class = get_top_words(ctf_idf, feature_names, top_n=10)

In [137]:
#turn top_words_per_class to a dataframe
top_words_per_class = pd.DataFrame(top_words_per_class)
top_words_per_class = top_words_per_class.T

top_words_per_class['new_words'] = top_words_per_class[0] + ', ' + top_words_per_class[1] + ', ' + top_words_per_class[2] + ', ' + top_words_per_class[3] + ', ' + top_words_per_class[4] + ', ' + top_words_per_class[5] + ', ' + top_words_per_class[6] + ', ' + top_words_per_class[7] + ', ' + top_words_per_class[8] + ', ' + top_words_per_class[9]

top_words_per_class = top_words_per_class.drop(columns=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
top_words_per_class

,new_words
0,"trip, us, factori, breakfast, love, noodl, aft..."
1,"massag, recommend, experi, great, siri, cook, ..."
2,"massag, recommend, amaz, relax, war, excel, we..."
3,"lee, spa, bridg, also, experi, market, quarter..."
4,"massag, servic, excel, recommend, good, amaz, ..."
...,...
2146,"place, month, ago, activ, find, want, went, vi..."
2147,"great, see, stroll, chang, overal, street, muc..."
2148,"hanoi, autumn, squid, cake, pho, sad, caramen,..."
2149,"comeback, enjoy, discov, drive, run, lake, aro..."


In [138]:
top_words_per_class['cluster'] = top_words_per_class.index

In [139]:
dtm_model = pd.merge(dtm_model, top_words_per_class, on='cluster', how='left')
dtm_model

,date,keywords,string,topic,cluster,frequency,Name,new_words
0,2023-06-01,None,"['famili', 'book', 'trip', 'market', 'via', 'o...",17,0,4,17_book_street_cafe_coffe,"trip, us, factori, breakfast, love, noodl, aft..."
1,2023-06-01,None,"['awesom', 'place', 'spend', 'half', 'day', 's...",-1,1,153,-1_place_walk_visit_street,"massag, recommend, experi, great, siri, cook, ..."
2,2023-06-01,None,"['will', 'spend', 'entranc', 'fee', 'plu', 'tr...",0,2,29,0_museum_war_histori_exhibit,"massag, recommend, amaz, relax, war, excel, we..."
3,2023-06-01,None,"['long', 'bridg', 'span', 'river', 'da', 'nang...",2,3,19,2_show_puppet_perform_opera,"lee, spa, bridg, also, experi, market, quarter..."
4,2023-06-01,None,"['quit', 'big', 'area', 'mani', 'sceneri', 'ph...",1,4,20,1_massag_spa_servic_staff,"massag, servic, excel, recommend, good, amaz, ..."
...,...,...,...,...,...,...,...,...
2146,2022-02-01,None,"['month', 'ago', 'went', 'place', 'nice', 'bea...",69,2146,1,69_street_cross_pavement_road,"place, month, ago, activ, find, want, went, vi..."
2147,2022-02-01,None,"['great', 'street', 'nice', 'stroll', 'see', '...",36,2147,1,36_bar_street_bui_vien,"great, see, stroll, chang, overal, street, muc..."
2148,2021-05-01,None,"['hanoi', 'old', 'quarter', 'good', 'place', '...",-1,2148,3,-1_place_walk_visit_street,"hanoi, autumn, squid, cake, pho, sad, caramen,..."
2149,2021-05-01,None,"['good', 'place', 'discov', 'enjoy', 'walk', '...",3,2149,1,3_pagoda_buddha_ladi_statu,"comeback, enjoy, discov, drive, run, lake, aro..."


In [140]:
import plotly.express as px

dtm_model['Name'] = dtm_model['Name'].astype(str)


pio.templates.default = "plotly_white"
fig = px.area(dtm_model, x="date", y="frequency", color="Name", hover_name='new_words', line_group="Name", color_discrete_sequence=px.colors.qualitative.Antique, height=600)
fig.update_traces(textposition="top right", textfont_color="black")

#whitegrid background

fig.update_layout(showlegend=True)

fig.show()